*Eduardus Tjitrahardja | @edutjie | 2022*

# Image Classification - VGGNet

## Importing Libraries

In [1]:
import os
# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation
from tensorflow.keras import regularizers, optimizers, Sequential
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

plt.style.use("seaborn")

In [2]:
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.9.1
/device:GPU:0
Num GPUs Available:  1


## Loading Data

In [2]:
IMG_SIZE = 224

In [3]:
def load_data(dir, labels = ["pumpkin", "tomato", "watermelon"],img_size = 224):
    data = []
    for label in labels:
        path = os.path.join(dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            img_path = os.path.join(path, img)
            img = cv2.imread(img_path)                      # load image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      # convert image from BGR to RGB format
            img = cv2.resize(img, (img_size, img_size))     # resize image to the preferred size
            img = np.array(img) / 255.0                     # normalize data
            data.append([img, class_num])
    return np.array(data)

In [4]:
train_data = load_data("images\\train")
val_data = load_data("images\\validation")

len(train_data), len(val_data)

C:\Users\edutjie\AppData\Local\Temp\ipykernel_10572\943813668.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


(3133, 824)

In [6]:
# target_df = pd.DataFrame(train_data[:, 1], columns = ["label"])
# target_df['label'] = target_df.label.map({0:'pumpkin', 1:'tomato', 2:'watermelon'})
# sns.countplot(x='label',data=target_df)
# plt.show()

In [7]:
# plt.imshow(train_data[0][0])
# plt.title(target_df.label[0])
# plt.show()

## Data Preprocessing

In [5]:
# seperate label and image array
train_img = []
train_labels = []

val_img = []
val_labels = []

for img, label in train_data:
    train_img.append(img)
    train_labels.append(label)

for img, label in val_data:
    val_img.append(img)
    val_labels.append(label)

# convert to np array
train_img = np.array(train_img)
train_labels = np.array(train_labels)
val_img = np.array(val_img)
val_labels = np.array(val_labels)

# # reshape data
# train_img = train_img.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
# val_img = val_img.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print(f"Train Image Shape: {train_img.shape}")
print(f"Train Labels Shape: {train_labels.shape}\n")
print(f"Validation Image Shape: {val_img.shape}")
print(f"Validation Labels Shape: {val_labels.shape}")

Train Image Shape: (3133, 224, 224, 3)
Train Labels Shape: (3133,)

Validation Image Shape: (824, 224, 224, 3)
Validation Labels Shape: (824,)


### Convert Data to Categorical

In [6]:
train_labels = keras.utils.to_categorical(train_labels, 3)
val_labels = keras.utils.to_categorical(val_labels, 3)

train_labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [10]:
print(f"Train Image Shape: {train_img.shape}")
print(f"Train Labels Shape: {train_labels.shape}\n")
print(f"Validation Image Shape: {val_img.shape}")
print(f"Validation Labels Shape: {val_labels.shape}")

Train Image Shape: (3133, 224, 224, 3)
Train Labels Shape: (3133, 3)

Validation Image Shape: (824, 224, 224, 3)
Validation Labels Shape: (824, 3)


## Define VGG Architecture

In [7]:
model = Sequential()
weight_decay = 0.0005
model_input = (IMG_SIZE, IMG_SIZE, 3)

model.add(Conv2D(64, (3, 3), padding='same', input_shape=model_input, kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 activation (Activation)     (None, 224, 224, 64)      0         
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 64)     256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 224, 224, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 activation_1 (Activation)   (None, 224, 224, 64)      0         
                                                        

In [8]:
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

## Training VVG

In [9]:
path = 'checkpoints\\model_vgg.hdf5'

# Keep only a single checkpoint, the best over test accuracy.
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, periode=5, mode='auto')

In [10]:
history = model.fit(train_img, train_labels, validation_data=(val_img, val_labels), epochs=5, batch_size=16, callbacks=[checkpoint])

Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node 'sequential/conv2d_2/Conv2D' defined at (most recent call last):
    File "d:\miniconda3\envs\tf\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\miniconda3\envs\tf\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "d:\miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\miniconda3\envs\tf\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "d:\miniconda3\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "d:\miniconda3\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "d:\miniconda3\envs\tf\lib\asyncio\base_events.py", line 600, in run_forever
      self._run_once()
    File "d:\mini